In [2]:
!pip install --quiet -U lancedb
!pip install --quiet gradio transformers torch torchvision


[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


In [14]:
data = tbl.to_lance().to_table()

In [30]:
import lance
import pyarrow.compute as pc

lance.write_dataset(data.filter(~pc.field("prompt").is_null()), "~/datasets/demo/diffusiondb.lance")

In [42]:
import io
import PIL
import duckdb
import lancedb

uri = "~/datasets/demo"
db = lancedb.connect(uri)
tbl = db.open_table("diffusiondb")
tbl.create_fts_index(["prompt"])

In [54]:
tbl.create_fts_index(["prompt"])

In [50]:
ds = tbl.to_lance()
duckdb.query("SELECT * FROM ds WHERE seed=3969484216").to_df()

,prompt,seed,step,cfg,sampler,width,height,timestamp,image_nsfw,prompt_nsfw,vector,image
0,portrait of jennifer lawrence as teenage mutan...,3969484216,50,7.0,k_lms,512,512,2022-08-16 19:37:00,0.108981,0.031935,"[0.1055411696434021, 0.16571573913097382, 0.09...","[137, 80, 78, 71, 13, 10, 26, 10, 0, 0, 0, 13,..."


In [55]:
tbl = db.open_table("diffusiondb")
tbl.search("ninja turtle").limit(9).to_df()

,prompt,seed,step,cfg,sampler,width,height,timestamp,image_nsfw,prompt_nsfw,vector,image,score
0,portrait of jennifer lawrence as teenage mutan...,3969484216,50,7.0,k_lms,512,512,2022-08-16 19:37:00,0.108981,0.031935,"[0.10554117, 0.16571574, 0.09008662, -0.194717...",b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\...,14.907752
1,fma stormtrooper and a ninja turtle having cof...,1391039028,50,7.0,k_lms,512,512,2022-08-18 11:14:00,0.194749,0.001279,"[-0.22904971, -0.023404881, 0.10390632, 0.5918...",b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\...,12.188478
2,a rainbow - shelled turtle by aardman entertai...,1135798550,50,7.0,k_lms,512,512,2022-08-18 09:04:00,0.036246,0.001061,"[0.3398647, 0.1279463, -0.113979734, -0.466874...",b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\...,9.189347
3,a rainbow - shelled turtle by aardman entertai...,253119891,50,7.0,k_lms,512,512,2022-08-18 09:05:00,0.038535,0.847200,"[0.2561665, -0.09101473, 0.097268306, -0.09523...",b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\...,8.676806
4,riven with blue eyes wearing zed mask and ninj...,2528242526,50,7.0,k_lms,512,512,2022-08-18 06:20:00,0.187990,0.031660,"[-0.22818531, 0.57053506, 0.058121443, 0.24126...",b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\...,7.319629
5,cute ninja cat and babymouse in a space statio...,3333593796,50,7.0,k_lms,512,512,2022-08-17 13:48:00,0.054645,0.000996,"[-0.1219545, 0.28735355, -0.320674, 0.30553597...",b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\...,6.117216
6,teenage mutant ninja turtles eat pizza in room...,2457359452,50,16.0,k_lms,512,512,2022-08-16 19:06:00,2.000000,0.324804,"[0.15923323, 0.1867303, -0.1716962, -0.1851373...",b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\...,5.173012
7,robot ninja mask terminator t 1 0 0 0 cyber ea...,1329918855,50,7.0,k_lms,512,512,2022-08-19 09:45:00,0.077714,0.000952,"[0.24278568, 0.109467536, -0.44478047, -0.1991...",b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\...,4.802383
8,cyborg robot ninja mask terminator t 1 0 0 0 f...,495303071,50,7.0,k_lms,512,512,2022-08-19 10:55:00,0.080432,0.001629,"[0.5367254, 0.337718, -0.6361712, 0.06989978, ...",b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\...,4.422182


ninja turtle


In [48]:
from transformers import CLIPModel, CLIPProcessor, CLIPTokenizerFast

MODEL_ID = "openai/clip-vit-base-patch32"

tokenizer = CLIPTokenizerFast.from_pretrained(MODEL_ID)
model = CLIPModel.from_pretrained(MODEL_ID)
processor = CLIPProcessor.from_pretrained(MODEL_ID)

def embed_func(query):
    inputs = tokenizer([query], padding=True, return_tensors="pt")
    text_features = model.get_text_features(**inputs)
    return text_features.detach().numpy()[0]

ninja turtle
turtle
turtle


In [46]:
def find_image_vectors(query):
    emb = embed_func(query)
    return _extract(tbl.search(emb).limit(9).to_df())

def find_image_keywords(query):
    print(query)
    return _extract(tbl.search(query).limit(9).to_df())

def find_image_sql(query):
    diffusiondb = tbl.to_lance()
    return _extract(duckdb.query(query).to_df())

def _extract(df):
    image_col = "image"
    return [(PIL.Image.open(io.BytesIO(row[image_col])), row["prompt"]) for _, row in df.iterrows()]

In [47]:
import gradio as gr


with gr.Blocks() as demo:

    with gr.Row():
        with gr.Tab("Embeddings"):
            vector_query = gr.Textbox(value="geodesic landscape", show_label=False)
            b1 = gr.Button("Submit")
        with gr.Tab("Keywords"):
            keyword_query = gr.Textbox(value="turtle", show_label=False)
            b2 = gr.Button("Submit")
        with gr.Tab("SQL"):
            sql_query = gr.Textbox(value="SELECT * from diffusiondb WHERE image_nsfw >= 2 LIMIT 9", show_label=False)
            b3 = gr.Button("Submit")
    with gr.Row():
        gallery = gr.Gallery(
                label="Found images", show_label=False, elem_id="gallery"
            ).style(columns=[3], rows=[3], object_fit="contain", height="auto")   
        
    b1.click(find_image_vectors, inputs=vector_query, outputs=gallery)
    b2.click(find_image_keywords, inputs=keyword_query, outputs=gallery)
    b3.click(find_image_sql, inputs=sql_query, outputs=gallery)
    
demo.launch()

Running on local URL:  http://127.0.0.1:7866

To create a public link, set `share=True` in `launch()`.


ninja turtle
